In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🗺️ US Counties Analytics Dashboard\n",
    "\n",
    "Interactive regional statistics with real US counties data and clean, modern design."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup and imports\n",
    "import pandas as pd\n",
    "import geopandas as gpd\n",
    "import folium\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "import numpy as np\n",
    "import requests\n",
    "import zipfile\n",
    "import os\n",
    "import ipywidgets as widgets\n",
    "from IPython.display import display, HTML, clear_output\n",
    "from shapely.geometry import Polygon\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "print('🚀 Setting up US Counties Dashboard...')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class SimpleCountiesDashboard:\n",
    "    \"\"\"Simple, working counties dashboard\"\"\"\n",
    "    \n",
    "    def __init__(self):\n",
    "        self.gdf = None\n",
    "        self.stats_data = None\n",
    "        \n",
    "        # Clean colors\n",
    "        self.colors = {\n",
    "            'primary': '#2D3748',\n",
    "            'surface': '#FFFFFF', \n",
    "            'text': '#1A202C',\n",
    "            'text_light': '#718096',\n",
    "            'border': '#E2E8F0',\n",
    "            'background': '#F7FAFC'\n",
    "        }\n",
    "        \n",
    "        # Outputs\n",
    "        self.header_output = widgets.Output()\n",
    "        self.controls_output = widgets.Output()\n",
    "        self.stats_output = widgets.Output()\n",
    "        self.map_output = widgets.Output()\n",
    "    \n",
    "    def download_counties_data(self):\n",
    "        \"\"\"Download real US counties or create fallback\"\"\"\n",
    "        print('📥 Attempting to download US Counties data...')\n",
    "        \n",
    "        try:\n",
    "            # Try to download real counties data\n",
    "            url = \"https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_20m.zip\"\n",
    "            filename = \"counties.zip\"\n",
    "            \n",
    "            if not os.path.exists(filename):\n",
    "                print('  Downloading from Census Bureau...')\n",
    "                response = requests.get(url, timeout=30)\n",
    "                with open(filename, 'wb') as f:\n",
    "                    f.write(response.content)\n",
    "                print(f'  ✅ Downloaded {len(response.content)/1024/1024:.1f}MB')\n",
    "            \n",
    "            # Extract\n",
    "            extract_dir = 'counties_data'\n",
    "            if not os.path.exists(extract_dir):\n",
    "                with zipfile.ZipFile(filename, 'r') as zip_ref:\n",
    "                    zip_ref.extractall(extract_dir)\n",
    "                print('  ✅ Extracted shapefile')\n",
    "            \n",
    "            # Load shapefile\n",
    "            shp_files = [f for f in os.listdir(extract_dir) if f.endswith('.shp')]\n",
    "            if shp_files:\n",
    "                shp_path = os.path.join(extract_dir, shp_files[0])\n",
    "                full_gdf = gpd.read_file(shp_path)\n",
    "                \n",
    "                # Filter to major states for performance\n",
    "                major_states = ['06', '48', '12', '36', '17', '42', '08', '04', '13', '37']\n",
    "                self.gdf = full_gdf[full_gdf['STATEFP'].isin(major_states)].copy()\n",
    "                self.gdf = self.gdf.to_crs(epsg=4326)\n",
    "                \n",
    "                print(f'  ✅ Loaded {len(self.gdf)} counties from major states')\n",
    "                return True\n",
    "                \n",
    "        except Exception as e:\n",
    "            print(f'  ⚠️ Download failed: {e}')\n",
    "        \n",
    "        # Fallback to sample data\n",
    "        print('  🔄 Using sample counties data...')\n",
    "        return self.create_sample_counties()\n",
    "    \n",
    "    def create_sample_counties(self):\n",
    "        \"\"\"Create sample county boundaries\"\"\"\n",
    "        \n",
    "        # Major US counties with realistic coordinates\n",
    "        counties_data = [\n",
    "            # California\n",
    "            ('Los Angeles County', 'CA', '06', (-118.9, 33.7, -117.6, 34.8)),\n",
    "            ('San Diego County', 'CA', '06', (-117.6, 32.5, -116.1, 33.5)),\n",
    "            ('Orange County', 'CA', '06', (-118.1, 33.3, -117.4, 33.9)),\n",
    "            ('Riverside County', 'CA', '06', (-117.7, 33.4, -114.1, 34.1)),\n",
    "            ('San Bernardino County', 'CA', '06', (-118.0, 34.0, -114.1, 35.8)),\n",
    "            \n",
    "            # Texas\n",
    "            ('Harris County', 'TX', '48', (-95.8, 29.5, -94.9, 30.1)),\n",
    "            ('Dallas County', 'TX', '48', (-97.0, 32.6, -96.4, 33.0)),\n",
    "            ('Tarrant County', 'TX', '48', (-97.5, 32.6, -96.9, 33.0)),\n",
    "            ('Bexar County', 'TX', '48', (-98.8, 29.1, -98.0, 29.8)),\n",
    "            ('Travis County', 'TX', '48', (-98.2, 30.0, -97.5, 30.5)),\n",
    "            \n",
    "            # Florida  \n",
    "            ('Miami-Dade County', 'FL', '12', (-80.9, 25.1, -80.1, 25.9)),\n",
    "            ('Broward County', 'FL', '12', (-80.6, 25.9, -80.0, 26.4)),\n",
    "            ('Palm Beach County', 'FL', '12', (-80.6, 26.3, -79.8, 27.0)),\n",
    "            ('Hillsborough County', 'FL', '12', (-82.8, 27.6, -82.0, 28.2)),\n",
    "            ('Orange County', 'FL', '12', (-81.7, 28.4, -81.1, 28.8)),\n",
    "            \n",
    "            # New York\n",
    "            ('New York County', 'NY', '36', (-74.1, 40.7, -73.9, 40.8)),\n",
    "            ('Kings County', 'NY', '36', (-74.1, 40.5, -73.8, 40.7)),\n",
    "            ('Queens County', 'NY', '36', (-73.9, 40.5, -73.7, 40.8)),\n",
    "            ('Suffolk County', 'NY', '36', (-73.3, 40.6, -71.9, 41.2)),\n",
    "            ('Nassau County', 'NY', '36', (-73.8, 40.6, -73.3, 40.8)),\n",
    "        ]\n",
    "        \n",
    "        geometries = []\n",
    "        names = []\n",
    "        states = []\n",
    "        statefps = []\n",
    "        \n",
    "        for name, state, statefp, (min_lon, min_lat, max_lon, max_lat) in counties_data:\n",
    "            coords = [\n",
    "                (min_lon, min_lat),\n",
    "                (max_lon, min_lat),\n",
    "                (max_lon, max_lat), \n",
    "                (min_lon, max_lat),\n",
    "                (min_lon, min_lat)\n",
    "            ]\n",
    "            \n",
    "            geometries.append(Polygon(coords))\n",
    "            names.append(name)\n",
    "            states.append(state)\n",
    "            statefps.append(statefp)\n",
    "        \n",
    "        self.gdf = gpd.GeoDataFrame({\n",
    "            'NAME': names,\n",
    "            'STUSPS': states,\n",
    "            'STATEFP': statefps,\n",
    "            'geometry': geometries\n",
    "        })\n",
    "        \n",
    "        print(f'  ✅ Created {len(self.gdf)} sample counties')\n",
    "        return True\n",
    "    \n",
    "    def generate_county_statistics(self):\n",
    "        \"\"\"Generate comprehensive synthetic statistics\"\"\"\n",
    "        print('📊 Generating synthetic county statistics...')\n",
    "        \n",
    "        counties = self.gdf['NAME'].tolist()\n",
    "        states = self.gdf['STUSPS'].tolist()\n",
    "        \n",
    "        # Generate 18 months of data\n",
    "        dates = pd.date_range(end=pd.Timestamp.now(), periods=18, freq='M')\n",
    "        \n",
    "        data_rows = []\n",
    "        \n",
    "        for county, state in zip(counties, states):\n",
    "            # Set realistic base values by county size/type\n",
    "            if 'Los Angeles' in county:\n",
    "                profile = {'pop': 10_000_000, 'gdp_pc': 70000, 'unemp_base': 5.5, 'type': 'major_metro'}\n",
    "            elif any(x in county for x in ['New York', 'Kings', 'Queens']):\n",
    "                profile = {'pop': 2_500_000, 'gdp_pc': 75000, 'unemp_base': 4.8, 'type': 'major_metro'}\n",
    "            elif any(x in county for x in ['Harris', 'Dallas', 'Cook', 'Miami-Dade']):\n",
    "                profile = {'pop': 4_000_000, 'gdp_pc': 65000, 'unemp_base': 5.2, 'type': 'metro'}\n",
    "            elif 'County' in county:\n",
    "                profile = {'pop': np.random.randint(500_000, 2_000_000), 'gdp_pc': np.random.uniform(50000, 70000), \n",
    "                          'unemp_base': np.random.uniform(4.5, 7.5), 'type': 'suburban'}\n",
    "            else:\n",
    "                profile = {'pop': np.random.randint(200_000, 1_000_000), 'gdp_pc': np.random.uniform(45000, 65000),\n",
    "                          'unemp_base': np.random.uniform(5.0, 8.0), 'type': 'small'}\n",
    "            \n",
    "            for i, date in enumerate(dates):\n",
    "                # Add realistic growth and variation\n",
    "                months_from_start = i\n",
    "                growth_factor = (1.01) ** (months_from_start / 12)  # 1% annual growth\n",
    "                seasonal_factor = 1 + 0.02 * np.sin(2 * np.pi * date.month / 12)  # 2% seasonal variation\n",
    "                economic_cycle = 1 + 0.05 * np.sin(2 * np.pi * months_from_start / 48)  # 4-year cycle\n",
    "                \n",
    "                current_pop = int(profile['pop'] * growth_factor * seasonal_factor)\n",
    "                current_gdp_pc = profile['gdp_pc'] * economic_cycle\n",
    "                \n",
    "                data_rows.append({\n",
    "                    'county': county,\n",
    "                    'state': state,\n",
    "                    'date': date,\n",
    "                    'county_type': profile['type'],\n",
    "                    \n",
    "                    # Core demographics\n",
    "                    'population': current_pop,\n",
    "                    'population_density': current_pop / np.random.uniform(100, 3000),\n",
    "                    'median_age': np.random.uniform(32, 45),\n",
    "                    'households': int(current_pop / np.random.uniform(2.2, 2.8)),\n",
    "                    \n",
    "                    # Age distribution\n",
    "                    'age_under_18': int(current_pop * np.random.uniform(0.18, 0.28)),\n",
    "                    'age_18_34': int(current_pop * np.random.uniform(0.20, 0.30)),\n",
    "                    'age_35_54': int(current_pop * np.random.uniform(0.25, 0.32)),\n",
    "                    'age_55_64': int(current_pop * np.random.uniform(0.12, 0.18)),\n",
    "                    'age_65_plus': int(current_pop * np.random.uniform(0.12, 0.22)),\n",
    "                    \n",
    "                    # Economics\n",
    "                    'gdp_millions': (current_pop * current_gdp_pc) / 1_000_000,\n",
    "                    'gdp_per_capita': current_gdp_pc,\n",
    "                    'median_income': current_gdp_pc * 0.75 * np.random.uniform(0.9, 1.1),\n",
    "                    'unemployment_rate': max(2.0, profile['unemp_base'] * (2.2 - economic_cycle) + np.random.uniform(-0.8, 0.8)),\n",
    "                    'poverty_rate': np.random.uniform(8, 20) * (2.5 - economic_cycle),\n",
    "                    'labor_force_participation': np.random.uniform(60, 75),\n",
    "                    \n",
    "                    # Housing\n",
    "                    'housing_units': int(current_pop / np.random.uniform(2.0, 2.6)),\n",
    "                    'median_home_value': current_gdp_pc * np.random.uniform(4, 9) * economic_cycle,\n",
    "                    'rent_burden_pct': np.random.uniform(25, 40),\n",
    "                    \n",
    "                    # Education\n",
    "                    'high_school_grad_rate': np.random.uniform(80, 96),\n",
    "                    'college_grad_rate': np.random.uniform(20, 50),\n",
    "                    'education_score': np.random.uniform(65, 92),\n",
    "                    \n",
    "                    # Health & Safety\n",
    "                    'life_expectancy': np.random.uniform(76, 82),\n",
    "                    'crime_rate_per_100k': np.random.uniform(150, 650),\n",
    "                    'violent_crime_rate': np.random.uniform(100, 400),\n",
    "                    \n",
    "                    # Infrastructure\n",
    "                    'broadband_access_pct': np.random.uniform(70, 95),\n",
    "                    'avg_commute_minutes': np.random.uniform(20, 40),\n",
    "                    'air_quality_index': np.random.uniform(35, 85),\n",
    "                    \n",
    "                    # Migration\n",
    "                    'net_migration': np.random.randint(-1000, 3000),\n",
    "                    'domestic_migration': np.random.randint(-500, 2000),\n",
    "                    'international_migration': np.random.randint(-200, 1500),\n",
    "                })\n",
    "        \n",
    "        self.stats_data = pd.DataFrame(data_rows)\n",
    "        \n",
    "        print(f'  ✅ Generated {len(self.stats_data):,} statistical records')\n",
    "        print(f'  📅 Time span: {dates[0].strftime(\"%Y-%m\")} to {dates[-1].strftime(\"%Y-%m\")}')\n",
    "        print(f'  🏘️ Counties: {len(counties)}')\n",
    "        print(f'  📊 Metrics: {len([col for col in self.stats_data.columns if col not in [\"county\", \"state\", \"date\", \"county_type\"]])}')\n",
    "        \n",
    "        return self.stats_data\n",
    "    \n",
    "    def create_dashboard_header(self):\n",
    "        \"\"\"Create clean dashboard header\"\"\"\n",
    "        with self.header_output:\n",
    "            header_html = f'''\n",
    "            <div style=\"\n",
    "                background: {self.colors['surface']};\n",
    "                padding: 50px 20px;\n",
    "                margin-bottom: 40px;\n",
    "                border-bottom: 1px solid {self.colors['border']};\n",
    "                text-align: center;\n",
    "            \">\n",
    "                <h1 style=\"\n",
    "                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "                    font-size: 2.8rem;\n",
    "                    font-weight: 300;\n",
    "                    color: {self.colors['text']};\n",
    "                    margin: 0 0 16px 0;\n",
    "                    letter-spacing: -0.02em;\n",
    "                \">US Counties Analytics</h1>\n",
    "                <p style=\"\n",
    "                    font-size: 1.2rem;\n",
    "                    color: {self.colors['text_light']};\n",
    "                    margin: 0 0 12px 0;\n",
    "                    font-weight: 400;\n",
    "                \">Interactive demographic and economic data visualization</p>\n",
    "                <p style=\"\n",
    "                    font-size: 0.95rem;\n",
    "                    color: {self.colors['text_light']};\n",
    "                    margin: 0;\n",
    "                \">🏘️ {len(self.gdf)} counties • 📅 18 months • 📈 25+ metrics</p>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(header_html))\n",
    "    \n",
    "    def create_interactive_controls(self):\n",
    "        \"\"\"Create dashboard controls\"\"\"\n",
    "        with self.controls_output:\n",
    "            # Get options\n",
    "            states = ['All States'] + sorted(self.stats_data['state'].unique().tolist())\n",
    "            counties = ['All Counties'] + sorted(self.stats_data['county'].unique().tolist())\n",
    "            \n",
    "            metrics = [\n",
    "                ('Population', 'population'),\n",
    "                ('GDP (Millions)', 'gdp_millions'),\n",
    "                ('GDP Per Capita', 'gdp_per_capita'),\n",
    "                ('Median Income', 'median_income'),\n",
    "                ('Unemployment Rate', 'unemployment_rate'),\n",
    "                ('Poverty Rate', 'poverty_rate'),\n",
    "                ('College Grad Rate', 'college_grad_rate'),\n",
    "                ('Crime Rate', 'crime_rate_per_100k'),\n",
    "                ('Life Expectancy', 'life_expectancy'),\n",
    "                ('Median Home Value', 'median_home_value'),\n",
    "                ('Broadband Access', 'broadband_access_pct'),\n",
    "                ('Air Quality Index', 'air_quality_index')\n",
    "            ]\n",
    "            \n",
    "            # Create widgets\n",
    "            self.state_dropdown = widgets.Dropdown(\n",
    "                options=states,\n",
    "                value='All States', \n",
    "                description='State:',\n",
    "                layout=widgets.Layout(width='180px')\n",
    "            )\n",
    "            \n",
    "            self.county_dropdown = widgets.Dropdown(\n",
    "                options=counties,\n",
    "                value='All Counties',\n",
    "                description='County:',\n",
    "                layout=widgets.Layout(width='220px')\n",
    "            )\n",
    "            \n",
    "            self.metric_dropdown = widgets.Dropdown(\n",
    "                options=metrics,\n",
    "                value='population',\n",
    "                description='Metric:',\n",
    "                layout=widgets.Layout(width='180px')\n",
    "            )\n",
    "            \n",
    "            self.view_dropdown = widgets.Dropdown(\n",
    "                options=[\n",
    "                    ('Map View', 'map'),\n",
    "                    ('Time Trends', 'trends'), \n",
    "                    ('Distribution', 'distribution'),\n",
    "                    ('Comparison', 'comparison')\n",
    "                ],\n",
    "                value='map',\n",
    "                description='View:',\n",
    "                layout=widgets.Layout(width='150px')\n",
    "            )\n",
    "            \n",
    "            # Update county list when state changes\n",
    "            def update_counties(change):\n",
    "                if change['new'] == 'All States':\n",
    "                    new_counties = ['All Counties'] + sorted(self.stats_data['county'].unique().tolist())\n",
    "                else:\n",
    "                    state_counties = self.stats_data[self.stats_data['state'] == change['new']]['county'].unique()\n",
    "                    new_counties = ['All Counties'] + sorted(state_counties.tolist())\n",
    "                \n",
    "                self.county_dropdown.options = new_counties\n",
    "                self.county_dropdown.value = 'All Counties'\n",
    "            \n",
    "            self.state_dropdown.observe(update_counties, names='value')\n",
    "            \n",
    "            # Update dashboard when controls change\n",
    "            def update_dashboard(*args):\n",
    "                self.update_visualizations()\n",
    "            \n",
    "            for widget in [self.state_dropdown, self.county_dropdown, self.metric_dropdown, self.view_dropdown]:\n",
    "                widget.observe(update_dashboard, names='value')\n",
    "            \n",
    "            # Controls layout\n",
    "            controls_html = f'''\n",
    "            <div style=\"\n",
    "                background: {self.colors['surface']};\n",
    "                padding: 25px;\n",
    "                margin-bottom: 30px;\n",
    "                border: 1px solid {self.colors['border']};\n",
    "                border-radius: 8px;\n",
    "            \">\n",
    "                <h3 style=\"\n",
    "                    margin: 0 0 20px 0;\n",
    "                    color: {self.colors['text']};\n",
    "                    font-size: 1.1rem;\n",
    "                    font-weight: 500;\n",
    "                \">Dashboard Controls</h3>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(controls_html))\n",
    "            \n",
    "            # Widget container\n",
    "            controls_container = widgets.HBox([\n",
    "                self.state_dropdown,\n",
    "                self.county_dropdown,\n",
    "                self.metric_dropdown,\n",
    "                self.view_dropdown\n",
    "            ], layout=widgets.Layout(justify_content='space-around'))\n",
    "            \n",
    "            display(controls_container)\n",
    "    \n",
    "    def create_summary_statistics(self):\n",
    "        \"\"\"Create summary stats cards\"\"\"\n",
    "        with self.stats_output:\n",
    "            # Get latest data\n",
    "            latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]\n",
    "            \n",
    "            # Filter by state if selected\n",
    "            if self.state_dropdown.value != 'All States':\n",
    "                latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]\n",
    "            \n",
    "            # Calculate stats\n",
    "            total_pop = latest_data['population'].sum()\n",
    "            avg_gdp_pc = latest_data['gdp_per_capita'].mean()\n",
    "            avg_unemployment = latest_data['unemployment_rate'].mean()\n",
    "            avg_income = latest_data['median_income'].mean()\n",
    "            avg_education = latest_data['college_grad_rate'].mean()\n",
    "            total_counties = len(latest_data)\n",
    "            \n",
    "            stats_html = f'''\n",
    "            <div style=\"\n",
    "                display: grid;\n",
    "                grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));\n",
    "                gap: 20px;\n",
    "                margin: 30px 0;\n",
    "            \">\n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {total_pop:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Total Population\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        ${avg_gdp_pc:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Avg GDP Per Capita\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {avg_unemployment:.1f}%\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Avg Unemployment\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        ${avg_income:,.0f}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Avg Median Income\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {avg_education:.1f}%\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        College Grad Rate\n",
    "                    </div>\n",
    "                </div>\n",
    "                \n",
    "                <div style=\"\n",
    "                    background: {self.colors['surface']};\n",
    "                    padding: 24px;\n",
    "                    border: 1px solid {self.colors['border']};\n",
    "                    border-radius: 8px;\n",
    "                    text-align: center;\n",
    "                \">\n",
    "                    <div style=\"font-size: 2rem; font-weight: 300; color: {self.colors['text']}; margin-bottom: 8px;\">\n",
    "                        {total_counties}\n",
    "                    </div>\n",
    "                    <div style=\"font-size: 0.875rem; color: {self.colors['text_light']}; font-weight: 500;\">\n",
    "                        Counties\n",
    "                    </div>\n",
    "                </div>\n",
    "            </div>\n",
    "            '''\n",
    "            display(HTML(stats_html))\n",
    "    \n",
    "    def create_map_visualization(self):\n",
    "        \"\"\"Create choropleth map\"\"\"\n",
    "        # Get latest data\n",
    "        latest_data = self.stats_data[self.stats_data['date'] == self.stats_data['date'].max()]\n",
    "        \n",
    "        # Filter by state if selected\n",
    "        if self.state_dropdown.value != 'All States':\n",
    "            latest_data = latest_data[latest_data['state'] == self.state_dropdown.value]\n",
    "            map_gdf = self.gdf[self.gdf['STUSPS'] == self.state_dropdown.value].copy()\n",
    "        else:\n",
    "            map_gdf = self.gdf.copy()\n",
    "        \n",
    "        # Merge with boundaries\n",
    "        map_data = map_gdf.merge(latest_data, left_on='NAME', right_on='county', how='left')\n",
    "        \n",
    "        # Create map\n",
    "        if len(map_data) > 0:\n",
    "            center_lat = map_data.geometry.centroid.y.mean()\n",
    "            center_lon = map_data.geometry.centroid.x.mean()\n",
    "            zoom_start = 6 if self.state_dropdown.value != 'All States' else 4\n",
    "            \n",
    "            m = folium.Map(\n",
    "                location=[center_lat, center_lon],\n",
    "                zoom_start=zoom_start,\n",
    "                tiles=None\n",
    "            )\n",
    "            \n",
    "            # Add clean tiles\n",
    "            folium.TileLayer(\n",
    "                tiles='https://{s}.basemaps.cartocdn.com/light_nopoi/{z}/{x}/{y}{r}.png',\n",
    "                attr='© CARTO',\n",
    "                name='Minimal',\n",
    "                overlay=False,\n",
    "                control=True\n",
    "            ).add_to(m)\n",
    "            \n",
    "            # Add choropleth\n",
    "            metric = self.metric_dropdown.value\n",
    "            if metric in map_data.columns and not map_data[metric].isna().all():\n",
    "                folium.Choropleth(\n",
    "                    geo_data=map_data.__geo_interface__,\n",
    "                    name=metric,\n",
    "                    data=latest_data,\n",
    "                    columns=['county', metric],\n",
    "                    key_on='feature.properties.NAME',\n",
    "                    fill_color='Greys',\n",
    "                    fill_opacity=0.7,\n",
    "                    line_opacity=0.3,\n",
    "                    line_color='#718096',\n",
    "                    line_weight=0.5,\n",
    "                    legend_name=dict(self.metric_dropdown.options)[metric]\n",
    "                ).add_to(m)\n",
    "                \n",
    "                # Add tooltips\n",
    "                for idx, row in map_data.iterrows():\n",
    "                    if pd.notna(row.get(metric)):\n",
    "                        popup_html = f'''\n",
    "                        <div style=\"\n",
    "                            padding: 16px;\n",
    "                            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "                            min-width: 200px;\n",
    "                        \">\n",
    "                            <h4 style=\"margin: 0 0 12px 0; color: #2D3748;\">{row['NAME']}</h4>\n",
    "                            <div style=\"color: #4A5568; font-size: 0.9rem;\">\n",
    "                                <strong>{dict(self.metric_dropdown.options)[metric]}:</strong><br>\n",
    "                                {row[metric]:,.1f}\n",
    "                            </div>\n",
    "                        </div>\n",
    "                        '''\n",
    "                        \n",
    "                        folium.GeoJson(\n",
    "                            row.geometry.__geo_interface__,\n",
    "                            popup=folium.Popup(popup_html, max_width=300),\n",
    "                            tooltip=row['NAME'],\n",
    "                            style_function=lambda x: {\n",
    "                                'fillOpacity': 0,\n",
    "                                'weight': 1,\n",
    "                                'color': '#718096'\n",
    "                            }\n",
    "                        ).add_to(m)\n",
    "            \n",
    "            return m\n",
    "        else:\n",
    "            # Empty map\n",
    "            return folium.Map(location=[39.8, -98.5], zoom_start=4)\n",
    "    \n",
    "    def create_chart_visualization(self):\n",
    "        \"\"\"Create chart visualizations\"\"\"\n",
    "        view_type = self.view_dropdown.value\n",
    "        metric = self.metric_dropdown.value\n",
    "        \n",
    "        # Filter data\n",
    "        data = self.stats_data.copy()\n",
    "        \n",
    "        if self.state_dropdown.value != 'All States':\n",
    "            data = data[data['state'] == self.state_dropdown.value]\n",
    "        \n",
    "        # Minimal chart styling\n",
    "        chart_layout = dict(\n",
    "            font=dict(family=\"-apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif\", color='#2D3748'),\n",
    "            plot_bgcolor='rgba(0,0,0,0)',\n",
    "            paper_bgcolor='rgba(0,0,0,0)',\n",
    "            margin=dict(l=50, r=40, t=60, b=40),\n",
    "            showlegend=False,\n",
    "            height=500\n",
    "        )\n",
    "        \n",
    "        if view_type == 'trends':\n",
    "            # Time series\n",
    "            if self.county_dropdown.value != 'All Counties':\n",
    "                trend_data = data[data['county'] == self.county_dropdown.value]\n",
    "                title = f\"{dict(self.metric_dropdown.options)[metric]} - {self.county_dropdown.value}\"\n",
    "            else:\n",
    "                trend_data = data.groupby('date')[metric].mean().reset_index()\n",
    "                title = f\"Average {dict(self.metric_dropdown.options)[metric]}\"\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Scatter(\n",
    "                x=trend_data['date'],\n",
    "                y=trend_data[metric],\n",
    "                mode='lines+markers',\n",
    "                line=dict(color='#4A5568', width=2),\n",
    "                marker=dict(size=4, color='#2D3748')\n",
    "            ))\n",
    "            \n",
    "        elif view_type == 'distribution':\n",
    "            # Histogram\n",
    "            latest_data = data[data['date'] == data['date'].max()]\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Histogram(\n",
    "                x=latest_data[metric],\n",
    "                nbinsx=15,\n",
    "                marker=dict(color='#718096', opacity=0.7)\n",
    "            ))\n",
    "            title = f\"Distribution of {dict(self.metric_dropdown.options)[metric]}\"\n",
    "            \n",
    "        elif view_type == 'comparison':\n",
    "            # Bar chart\n",
    "            latest_data = data[data['date'] == data['date'].max()]\n",
    "            top_data = latest_data.nlargest(12, metric)\n",
    "            \n",
    "            fig = go.Figure()\n",
    "            fig.add_trace(go.Bar(\n",
    "                x=top_data[metric],\n",
    "                y=top_data['county'],\n",
    "                orientation='h',\n",
    "                marker=dict(color='#4A5568')\n",
    "            ))\n",
    "            title = f\"Top Counties - {dict(self.metric_dropdown.options)[metric]}\"\n",
    "        \n",
    "        fig.update_layout(\n",
    "            title=dict(text=title, font=dict(size=16)),\n",
    "            **chart_layout\n",
    "        )\n",
    "        \n",
    "        return fig\n",
    "    \n",
    "    def update_visualizations(self):\n",
    "        \"\"\"Update all visualizations\"\"\"\n",
    "        with self.map_output:\n",
    "            clear_output(wait=True)\n",
    "            \n",
    "            if self.view_dropdown.value == 'map':\n",
    "                map_obj = self.create_map_visualization()\n",
    "                display(map_obj._repr_html_())\n",
    "            else:\n",
    "                fig = self.create_chart_visualization()\n",
    "                fig.show()\n",
    "        \n",
    "        # Update summary stats\n",
    "        self.create_summary_statistics()\n",
    "    \n",
    "    def launch_dashboard(self):\n",
    "        \"\"\"Launch the complete dashboard\"\"\"\n",
    "        print('\\n🚀 Launching US Counties Analytics Dashboard')\n",
    "        print('=' * 50)\n",
    "        \n",
    "        # Load data\n",
    "        print('\\n📂 Loading county boundaries...')\n",
    "        self.download_counties_data()\n",
    "        \n",
    "        print('\\n📊 Generating statistics...')\n",
    "        self.generate_county_statistics()\n",
    "        \n",
    "        print('\\n🎨 Building interface...')\n",
    "        self.create_dashboard_header()\n",
    "        self.create_interactive_controls()\n",
    "        self.create_summary_statistics()\n",
    "        \n",
    "        print('\\n🗺️ Rendering initial visualization...')\n",
    "        with self.map_output:\n",
    "            self.update_visualizations()\n",
    "        \n",
    "        # Apply clean styling\n",
    "        style_html = '''\n",
    "        <style>\n",
    "        .widget-dropdown select {\n",
    "            border: 1px solid #E2E8F0;\n",
    "            border-radius: 6px;\n",
    "            padding: 10px 12px;\n",
    "            background: #FFFFFF;\n",
    "            color: #1A202C;\n",
    "            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;\n",
    "            font-size: 0.875rem;\n",
    "        }\n",
    "        .widget-dropdown select:focus {\n",
    "            outline: none;\n",
    "            border-color: #718096;\n",
    "            box-shadow: 0 0 0 3px rgba(113,128,150,0.1);\n",
    "        }\n",
    "        .widget-hbox {\n",
    "            display: flex;\n",
    "            gap: 25px;\n",
    "            align-items: flex-end;\n",
    "            flex-wrap: wrap;\n",
    "            justify-content: space-around;\n",
    "        }\n",
    "        </style>\n",
    "        '''\n",
    "        display(HTML(style_html))\n",
    "        \n",
    "        # Create main layout\n",
    "        dashboard_layout = widgets.VBox([\n",
    "            self.header_output,\n",
    "            self.controls_output,\n",
    "            self.stats_output,\n",
    "            self.map_output\n",
    "        ])\n",
    "        \n",
    "        display(dashboard_layout)\n",
    "        \n",
    "        print('\\n✅ Dashboard launched successfully!')\n",
    "        print('🎯 Use the controls above to explore county data')\n",
    "        print('🗺️ Try different states, metrics, and visualization types')\n",
    "\n",
    "# Initialize dashboard\n",
    "dashboard = SimpleCountiesDashboard()\n",
    "print('Dashboard class initialized ✅')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Launch the dashboard\n",
    "dashboard.launch_dashboard()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 }
}